In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os
os.chdir("./..")
import PARC.data.data_em as data
#from parc import misc, metrics, visualization
from PARC.model import model_em as model
from PARC import misc

2024-06-20 12:03:40.058564: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-20 12:03:40.062047: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-20 12:03:40.107912: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-20 12:03:41.500366: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-06-20 12:03:43.671865: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/in

# Data pipeline

In [ ]:
# Get data and normalization
data_em = data.DataEnergeticMaterials()
state_seq_whole, vel_seq_whole = data_em.clip_raw_data(dataset_range= (0,150), dir_dataset= "../data/em/") # Get void_0 - void_150 for training
state_seq_norm = data_em.data_normalization(state_seq_whole,3) # Normalizing for state variables
vel_seq_norm = data_em.data_normalization(vel_seq_whole,2) # Normalizing for velocity variables

# Training


### Stage 1: Differentiator training

In [3]:
# Create tf.dataset
dataset_input = tf.data.Dataset.from_tensor_slices((state_seq_norm[0][:,:,:,:3],vel_seq_norm[0][:,:,:,:2]))
dataset_label = tf.data.Dataset.from_tensor_slices((state_seq_norm[0][:,:,:,-3:],vel_seq_norm[0][:,:,:,-2:]))
dataset = tf.data.Dataset.zip((dataset_input, dataset_label))
dataset = dataset.shuffle(buffer_size = 2192) 
dataset = dataset.batch(8)

2024-02-01 17:39:23.206071: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1636] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46594 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:81:00.0, compute capability: 8.6


In [ ]:
tf.keras.backend.clear_session()
parc = model.PARCv2(n_state_var = 3, n_time_step = 1, step_size= 1/15, m_input_shape = (128,192), solver = "rk4", mode = "differentiator_training")
# parc.differentiator.load_weights('parc2_diff_rk4.h5')
parc.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.00001, beta_1 = 0.9, beta_2 = 0.999))
parc.fit(dataset, epochs = 500, shuffle = True)

In [ ]:
parc.differentiator.save_weights('PARC2_EM_diff_rk4.h5')

### Stage 2: Data-driven integration training

In [ ]:
# Pretrain integrator
tf.keras.backend.clear_session()
parc = model.PARCv2(n_state_var = 3, n_time_step = 1, step_size= 1/15, m_input_shape = (128,192), solver = "rk4", mode = "integrator_training")
parc.differentiator.load_weights('parc2_diff_rk4.h5')
parc.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.00001, beta_1 = 0.9, beta_2 = 0.999))
parc.fit(dataset, epochs = 500, shuffle = True)

In [ ]:
parc.integrator.save_weights('PARC2_EM_int_rk4.h5')

# Validation

In [ ]:
state_seq_whole, vel_seq_whole = data.clip_raw_data(idx_range = (150,200), sequence_length = 15, n_state_var = 3, purpose = "test") # Get void_150 - void_200 for testing
state_seq_norm_test = data.data_normalization_test(state_seq_whole, state_seq_norm[1], state_seq_norm[2],3) # Normalization for state based on training values
vel_seq_norm_test = data.data_normalization_test(vel_seq_whole, vel_seq_norm[1], vel_seq_norm[2],2) # Normalization for velocity based on training values

## Load model

In [ ]:
# Load model for testing, set the number of time step to 15 (predict 15 steps)
tf.keras.backend.clear_session()
parc = model.PARCv2(n_state_var = 3, n_time_step = 15, step_size= 1/15, solver = "rk4")
parc.compile()

# Load pretrained weights
misc.geturl(url="https://virginia.box.com/shared/static/ziwd2we3z4mvhkcw4k9uj675fe3k4i92", title="PARC2_EM_diff_rk4.h5")
misc.geturl(url="https://virginia.box.com/shared/static/410wh4hy4gzuwwjlkrui628q28dsmqxn", title="PARC2_EM_int_rk4.h5")

parc.differentiator.load_weights('PARC2_EM_diff_rk4.h5')
parc.integrator.load_weights('PARC2_EM_int_rk4.h5')

In [ ]:
# Make prediction for the whole 34 test data
pred_whole = []
# outlier = [0, 3, 30, 31, 32, 33, 18, 19, 11]
# outlier = []
for idx in range(34):
    state_var_current = state_seq_norm_test[idx:idx+1,:,:,0:3]
    velocity_current = vel_seq_norm_test[idx:idx+1,:,:,0:2]
    pred_state = parc.predict([state_var_current,velocity_current])
    pred_state_case = np.concatenate(pred_state, axis = -1)
    pred_whole.append(pred_state_case)
pred_whole = np.concatenate(pred_whole, axis = 0)
print(pred_whole.shape)       

In [ ]:
# Denormalizing prediction data

pred_out = data.data_denormalization(pred_whole,state_seq_norm[1] + vel_seq_norm[1],state_seq_norm[2] + vel_seq_norm[2], no_of_channel = 5)

In [ ]:
# Save results file for plotting
np.save('./results/em/parcv2_em.npy',pred_out)

## Visualizing result

In [ ]:
# Load data
gt = np.load('../result/em/gt.npy')
# NeuralODE
parc = np.load('../result/em/neuralode_em.npy')
# NeuralODE
parc_data_int = np.load('../result/em/neuralode_em_pure_data_driven.npy')
# FNO
fno = np.load('../result/em/fno_01.npy')
# PARCv2
parcv2 = np.load('../result/em/parcv2_em.npy')

y_pred = [parcv2, parc, parc_data_int[:,:,:,:-5], fno]


In [ ]:
# Plot field evolution for tempereature
from PARC.visualization import visualization_em as viz

viz.plot_field_evolution(y_pred, gt, 28, options = 'temperature')

In [ ]:
# Plot field evolution for pressure
viz.plot_field_evolution(y_pred, gt, 28, options = 'pressure')